In [ ]:
import sys
sys.path.append("/Users/user/question-retrieval-KIPerWeb/")
from utils import *
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
import json
from nltk.stem.cistem import Cistem
from nltk.corpus import stopwords
from ranx import Qrels, evaluate, Run
import swifter
import json
import requests
from sklearn.preprocessing import LabelEncoder
from trectools import TrecPoolMaker, TrecRun
from tqdm import tqdm
from collections import Counter
import spacy
from sentence_transformers import SentenceTransformer, util, CrossEncoder
from operator import itemgetter
import joblib
import numpy as np

## Read the QB's sample

In [ ]:
# Read preprocessed data
# qb = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/samples/qb_stratified-sample.csv").fillna("N/A")
# qb = qb.drop(columns=['docid'])
# qb.rename(columns={"category":"topic_label_de_fixed"}, inplace=True)
# qb['docid'] = qb.index
# qb['content'] = qb['content'].apply(lambda x:remove_stop_words(x, 'german'))


# Preprocessing
# Models for preprocessing
nlp = spacy.load("de_core_news_sm")
tagger_de = ht.HanoverTagger('morphmodel_ger.pgz')

# qb['content'] = qb['content'].apply(lambda x : preprocess_documents(x, tagger_de))
# qb

## Calculate Embeddings and Initialize NMSLIB Indices

In [ ]:
# model_1 = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
# model_2 = SentenceTransformer("aari1995/German_Semantic_STS_V2")
# model_3 = SentenceTransformer("sentence-transformers/LaBSE")
# model_4 = SentenceTransformer("PM-AI/bi-encoder_msmarco_bert-base_german")
# model_5 = SentenceTransformer("efederici/e5-base-multilingual-4096")
# model_6 = SentenceTransformer("intfloat/multilingual-e5-base")
# model_7 = SentenceTransformer("clips/mfaq")
# model_8 = SentenceTransformer("PM-AI/sts_paraphrase_xlm-roberta-base_de-en")
# model_9 = SentenceTransformer("deutsche-telekom/gbert-large-paraphrase-euclidean")
# model_10 = SentenceTransformer("LLukas22/all-MiniLM-L12-v2-embedding-all")
# model_11 = SentenceTransformer("LLukas22/paraphrase-multilingual-mpnet-base-v2-embedding-all")
# model_12 = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v1")
# model_13 = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")
# model_14 = SentenceTransformer("deutsche-telekom/gbert-large-paraphrase-cosine")
# model_15 = SentenceTransformer("shibing624/text2vec-base-multilingual")
# model_16 = SentenceTransformer("Sahajtomar/German-semantic")
# model_17 = SentenceTransformer("setu4993/LaBSE")
# model_18 = SentenceTransformer("symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli")
# model_19 = SentenceTransformer("and-effect/musterdatenkatalog_clf")
# model_20 = SentenceTransformer("nblokker/debatenet-2-cat")
# model_21 = SentenceTransformer("setu4993/LEALLA-large")
# model_22 = SentenceTransformer("dell-research-harvard/lt-wikidata-comp-de")
# model_23 = SentenceTransformer("ef-zulla/e5-multi-sml-torch")
# model_24 = SentenceTransformer("barisaydin/text2vec-base-multilingual")
# model_25 = SentenceTransformer("meta-llama/Llama-2-7b-chat-hf")

In [ ]:
# # # # # Create the indices

# # # index_1 = create_index(qb, model_1, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-paraphrase-multilingual-mpnet-base-v2.nmslib")
# # # index_2 = create_index(qb, model_2, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/aari1995-German_Semantic_STS_V2.nmslib")
# # # index_3 = create_index(qb, model_3, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-LaBSE.nmslib")
# # # index_4 = create_index(qb, model_4, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/PM-AI-bi-encoder_msmarco_bert-base_german.nmslib")
# # # index_5 = create_index(qb, model_5, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/efederici-e5-base-multilingual-4096.nmslib")
# # # index_6 = create_index(qb, model_6, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/intfloat-multilingual-e5-base.nmslib")
# # # index_7 = create_index(qb, model_7, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/clips-mfaq.nmslib")
# # # index_8 = create_index(qb, model_8, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/PM-AI-sts_paraphrase_xlm-roberta-base_de-en.nmslib")
# # # index_9 = create_index(qb, model_9, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/deutsche-telekom-gbert-large-paraphrase-euclidean.nmslib")
# # # index_10 = create_index(qb, model_10, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/LLukas22-all-MiniLM-L12-v2-embedding-all.nmslib")
# # # index_11 = create_index(qb, model_11, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/LLukas22-paraphrase-multilingual-mpnet-base-v2-embedding-all.nmslib")
# # # index_12 = create_index(qb, model_12, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-distiluse-base-multilingual-cased-v1.nmslib")
# # # index_13 = create_index(qb, model_13, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-distiluse-base-multilingual-cased-v2.nmslib")
# # # index_14 = create_index(qb, model_14, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/deutsche-telekom-gbert-large-paraphrase-cosine.nmslib")
# # # index_15 = create_index(qb, model_15, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/shibing624-text2vec-base-multilingual.nmslib")
# # # index_16 = create_index(qb, model_16, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/Sahajtomar-German-semantic.nmslib")
# # # index_17 = create_index(qb, model_17, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/setu4993-LaBSE.nmslib")
# # # index_18 = create_index(qb, model_18, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/symanto-sn-xlm-roberta-base-snli-mnli-anli-xnli.nmslib")
# # # index_19 = create_index(qb, model_19, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/and-effect-musterdatenkatalog_clf.nmslib")
# # # index_20 = create_index(qb, model_20, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/nblokker-debatenet-2-cat.nmslib")
# # # index_21 = create_index(qb, model_21, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/setu4993-LEALLA-large.nmslib")
# # # index_22 = create_index(qb, model_22, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/dell-research-harvard-lt-wikidata-comp-de.nmslib")
# # # index_23 = create_index(qb, model_23, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/ef-zulla-e5-multi-sml-torch.nmslib")
# # # index_24 = create_index(qb, model_24, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/barisaydin-text2vec-base-multilingual.nmslib")
# # # index_25 = create_index(qb, model_25, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices/meta-llama-Llama-2-7b-chat-hf.nmslib")


# # # # Or load them:

# index_1 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-paraphrase-multilingual-mpnet-base-v2.nmslib")
# index_2 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/aari1995-German_Semantic_STS_V2.nmslib")
# index_3 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-LaBSE.nmslib")
# index_4 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/PM-AI-bi-encoder_msmarco_bert-base_german.nmslib")
# index_5 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/efederici-e5-base-multilingual-4096.nmslib")
# index_6 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/intfloat-multilingual-e5-base.nmslib")
# index_7 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/clips-mfaq.nmslib")
# index_8 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/PM-AI-sts_paraphrase_xlm-roberta-base_de-en.nmslib")
# index_9 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/deutsche-telekom-gbert-large-paraphrase-euclidean.nmslib")
# index_10 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/LLukas22-all-MiniLM-L12-v2-embedding-all.nmslib")
# index_11 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/LLukas22-paraphrase-multilingual-mpnet-base-v2-embedding-all.nmslib")
# index_12 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-distiluse-base-multilingual-cased-v1.nmslib")
# index_13 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/sentence-transformers-distiluse-base-multilingual-cased-v2.nmslib")
# index_14 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/deutsche-telekom-gbert-large-paraphrase-cosine.nmslib")
# index_15 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/shibing624-text2vec-base-multilingual.nmslib")
# index_16 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/Sahajtomar-German-semantic.nmslib")
# index_17 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/setu4993-LaBSE.nmslib")
# index_18 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/symanto-sn-xlm-roberta-base-snli-mnli-anli-xnli.nmslib")
# index_19 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/and-effect-musterdatenkatalog_clf.nmslib")
# index_20 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/nblokker-debatenet-2-cat.nmslib")
# index_21 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/setu4993-LEALLA-large.nmslib")
# index_22 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/dell-research-harvard-lt-wikidata-comp-de.nmslib")
# index_23 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/ef-zulla-e5-multi-sml-torch.nmslib")
# index_24 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/barisaydin-text2vec-base-multilingual.nmslib")
# index_25 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices/meta-llama-Llama-2-7b-chat-hf.nmslib")


## Run Queries Against Search Indices

In [ ]:
# query_test = search('Gesundheitswesen'.lower(), index_25, model_25, qb, k=100)['search_output']
# query_test

## Load the topics from the BFZ-EdTec Dataset

In [ ]:
## We could use queries:
# queries_file = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/topics_experiments/topics/topics.csv")
queries_file = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/queries_experiments/queries/queries.csv")
queries_file['queries'] = queries_file['queries'].apply(clean_text)



print("-> Number of queries:",len(queries_file['queries'].tolist()))

queries = queries_file['queries'].tolist()
query_ids = queries_file['qid'].tolist()


In [ ]:



def rerank_documents(xgboost_model, sentence_model, doc_query_pairs):
    # Load the XGBoost model

    # Initialize the list for features and original data
    features = []
    original_data = []

    for doc_id, query, document in doc_query_pairs:
        # Convert query and document to embeddings
        query_embedding = sentence_model.encode(query, convert_to_tensor=False)
        doc_embedding = sentence_model.encode(document, convert_to_tensor=False)

        # Create a feature vector
        feature_vector = np.concatenate([query_embedding, doc_embedding])
        features.append(feature_vector)

        # Store the original data
        original_data.append((doc_id, query, document))

    # Convert the list of features to a numpy array
    features_array = np.array(features)

    # Predict rankings
    predictions = xgboost_model.predict(features_array)

    # Combine predictions with the original data
    results = [(doc_id, query, document, score) for (doc_id, query, document), score in zip(original_data, predictions)]

    return results

# # Example usage
# xgboost_model_path = 'path_to_your_model.model'
# sentence_model = SentenceTransformer('your_model_name')
# doc_query_pairs = [('doc1_id', 'query1', 'document1'), ('doc2_id', 'query2', 'document2'), ...]  # Replace with your data

# for result in ranked_results:
#     print(result)


# # rerank_documents(xgboost_model_path, sentence_model, qb['content'].tolist()[:100], queries[0])

## Generate searches and store them as JSON files

In [ ]:
# path = '/Users/user/question-retrieval-KIPerWeb/runs/ANN_CrossEncoder-Re-Ranker/json_format/'


# for i, j in zip(queries, query_ids):
#     print(i, j)
#     search(i, index_1, model_1, qb, k=100, run_path=f'{path}query_{j}/run_01.json')
#     search(i, index_2, model_2, qb, k=100, run_path=f'{path}query_{j}/run_02.json')
#     search(i, index_3, model_3, qb, k=100, run_path=f'{path}query_{j}/run_03.json')
#     search(i, index_4, model_4, qb, k=100, run_path=f'{path}query_{j}/run_04.json')
#     search(i, index_5, model_5, qb, k=100, run_path=f'{path}query_{j}/run_05.json')
#     search(i, index_6, model_6, qb, k=100, run_path=f'{path}query_{j}/run_06.json')
#     search(i, index_7, model_7, qb, k=100, run_path=f'{path}query_{j}/run_07.json')
#     search(i, index_8, model_8, qb, k=100, run_path=f'{path}query_{j}/run_08.json')
#     search(i, index_9, model_9, qb, k=100, run_path=f'{path}query_{j}/run_09.json')
#     search(i, index_10, model_10, qb, k=100, run_path=f'{path}query_{j}/run_10.json')
#     search(i, index_11, model_11, qb, k=100, run_path=f'{path}query_{j}/run_11.json')
#     search(i, index_12, model_12, qb, k=100, run_path=f'{path}query_{j}/run_12.json')
#     search(i, index_13, model_13, qb, k=100, run_path=f'{path}query_{j}/run_13.json')
#     search(i, index_14, model_14, qb, k=100, run_path=f'{path}query_{j}/run_14.json')
#     search(i, index_15, model_15, qb, k=100, run_path=f'{path}query_{j}/run_15.json')
#     search(i, index_16, model_16, qb, k=100, run_path=f'{path}query_{j}/run_16.json')
#     search(i, index_17, model_17, qb, k=100, run_path=f'{path}query_{j}/run_17.json')
#     search(i, index_18, model_18, qb, k=100, run_path=f'{path}query_{j}/run_18.json')
#     search(i, index_19, model_19, qb, k=100, run_path=f'{path}query_{j}/run_19.json')
#     search(i, index_20, model_20, qb, k=100, run_path=f'{path}query_{j}/run_20.json')
#     search(i, index_21, model_21, qb, k=100, run_path=f'{path}query_{j}/run_21.json')
#     search(i, index_22, model_22, qb, k=100, run_path=f'{path}query_{j}/run_22.json')
#     search(i, index_23, model_23, qb, k=100, run_path=f'{path}query_{j}/run_23.json')
#     search(i, index_24, model_24, qb, k=100, run_path=f'{path}query_{j}/run_24.json')
#     search(i, index_25, model_25, qb, k=100, run_path=f'{path}query_{j}/run_25.json')    

## Preprocess all the synthetic searches and turn them into TREC style files

In [ ]:
# List all the JSON outputs of each run

path = '/Users/user/question-retrieval-KIPerWeb/runs/ANN_LM/json_format/'
json_files = list_files_from_dir(path, extention='.json')
# json_files

In [ ]:
# We check the size of the sessions
file = "/Users/user/question-retrieval-KIPerWeb/runs/ANN_LM/json_format/query_12/deutsche-telekom_gbert-large-paraphrase-cosine_run_14.json"
# assert 200 == len(read_json(file))

## LambdaMart Re-ranker

In [ ]:
def reorder_ranked_list(run, order):
    id2dict = dict((d['docid'], d) for d in run)
    re_ranked = [id2dict[x] for x in order]
    return re_ranked



xgboost_model = joblib.load('/Users/user/question-retrieval-KIPerWeb/notebooks/ground_truth_acquisition/building_training_dataset/final_xgbranker_model.pkl')
model = SentenceTransformer('LLukas22/paraphrase-multilingual-mpnet-base-v2-embedding-all')


def re_rank(path, model):
    run = read_json(path)
    re_ranked_elements = []
    doc_ids = []
    docs = []
    queries = []
    for i, j in enumerate(run):
        df = pd.Series(j,index=j.keys()).to_frame()
        content = ' '.join(df.T['text'].tolist() + df.T['correct_answers_temp'].tolist()) # -> fix this in the other cross encoder re-ranker
        content = preprocess_documents(content, tagger_de)
        query = df.T['query'].tolist()[0]
#         print(f"Query: {query}\nContent: {content}")
#         print("----------------------------------------")
        doc_ids.append(df.T['docid'].tolist()[0])
        docs.append(content)
        queries.append(query)

    docs = list(zip(doc_ids, queries, docs))
    
    origial_order = [e[0] for e in docs]
    
#     print(f"original order: {origial_order}")
    
    scored_data = rerank_documents(xgboost_model, model, docs)
    scored_data.sort(key=lambda x: x[3], reverse=True)
    re_ranked_list = scored_data
    order = [e[0] for e in re_ranked_list]
#     print(f"reordered {order}")
    re_ordered_list = reorder_ranked_list(run, order)
    path1 = list(Path(path).parts)[-2:]
    path1.insert(1, 'LambdaMart-')
    path1 = tuple(path1)
    path1 = (path1[0], path1[1] + path1[2])
    path1 = '/'.join(path1)
    path2 = '/Users/user/question-retrieval-KIPerWeb/runs/ANN_LambdaMArt-Re-Ranker/json_format/'
    path3 = path2+path1
    return save_dict_as_json(reorder_ranked_list(run, order), path3)


for e in tqdm(json_files):
    re_rank(e, model)

In [ ]:
# List all the JSON outputs of each run

path = '/Users/user/question-retrieval-KIPerWeb/runs/ANN_LambdaMArt-Re-Ranker/json_format/'
json_files = list_files_from_dir(path, extention='.json')
# json_files

In [ ]:
# For each JSON output, transform the results to a TREC format

def to_trec_format(a_path):
    if Path(a_path).suffix == '.json':
        results = read_json(a_path)
        ids = []
        query = []
        similarity = []
        for i,j in enumerate(results):
            ids.append(j['docid'])
            query.append(j['query'])
            try:
                similarity.append(j['cosine_distance'])
            except KeyError:
                similarity.append(j['BM25_score'])
        a_df = pd.DataFrame({'docno':ids, 'qid':query,'score':similarity})
        #         model_name = str('-'.join(Path(e).parts[-1:][0].split('_')[2:-2]))
        model_name = list(Path(a_path).parts)[-1:][0]
        model_name = [e for e in model_name.split('.') if e!='json']
        model_name = ['_'.join(e.split("_")[:-1][:-1]) for e in model_name][0]
        

        a_df['tag'] = model_name
        a_df['Q0'] = 'Q0'
        a_df['rank'] = a_df.index+1
        a_df = a_df[['qid', 'Q0', 'docno', 'rank', 'score', 'tag']]
        
        tsv_path = Path(a_path).with_suffix('.tsv')
        return a_df.to_csv(tsv_path, index = False, sep = '\t', header=False)
    else:
        pass



for e in json_files:
    try:
        to_trec_format(e)
    except IndexError:
        print(f"Error! in {e}")

In [ ]:
#list the TREC formatted files, these have a .tsv extention
list_files_from_dir(path, extention='.tsv')

# Process all the runs and continue with the trec formatting
path_list_processed = list_files_from_dir(path, extention='.tsv')

list_of_dfs = []
for e in path_list_processed:
    df = pd.concat([pd.read_csv(e, header=None, sep='\t', names=["queries", "Q0", "docid", "rank", "score", "tag"])])
    list_of_dfs.append(df)
    


all_runs = pd.concat(list_of_dfs).reset_index(drop=True)


all_runs= all_runs.merge(queries_file,on='queries')
all_runs = all_runs[["qid", "Q0", "docid", "score", "rank", "tag"]]

all_runs['qid'] = all_runs['qid'].astype(str)
all_runs['docid'] = all_runs['docid'].astype(str)

all_runs

In [ ]:
# Extract runs by different LMs
queries = list(set(all_runs['qid']))
L_models = list(set(all_runs['tag']))
runs = [all_runs.query(f'tag=="{e}"') for e in L_models]

for i, j in zip(runs, L_models):
    i.to_csv(f'/Users/user/question-retrieval-KIPerWeb/runs/ANN_LambdaMArt-Re-Ranker/trec_format/{j}_run.txt', header = None, sep='\t',index=False )
#     i.to_csv(f'/Users/user/question-retrieval-KIPerWeb/runs/ANN_CrossEncoder-Re-Ranker/trec_format/{j}_run.txt',index=False )
    

In [ ]:
f = "/Users/user/question-retrieval-KIPerWeb/testbeds/queries_experiments/trec_pools/testbed.csv"

qrels_df = pd.read_csv(f)
qrels_df['qid'] = qrels_df['qid'].astype(str)
qrels_df['docid'] = qrels_df['docid'].astype(str)

qrels = Qrels.from_df(
    df=qrels_df,
    q_id_col="qid",
    doc_id_col="docid",
    score_col="qrel",
)

In [ ]:



path = '/Users/user/question-retrieval-KIPerWeb/runs/ANN_LambdaMArt-Re-Ranker/trec_format/'
trec_runs = list_files_from_dir(path, extention='.txt')
trec_runs

runs = []
for e in trec_runs:
    runs.append(Run.from_file(e))
    
runs_names = []
for e in trec_runs:
    runs_names.append(get_last_element_of_path(e))



In [ ]:
report = create_report(qrels, runs, runs_names, 25,"/Users/user/question-retrieval-KIPerWeb/data/results/LambdaMart_Re-ranker/ann_LambdaMart_at_25.csv")
report.sort_values(by='mrr@25',  ascending=False).head(5)

In [ ]:
report = create_report(qrels, runs, runs_names, 50,"/Users/user/question-retrieval-KIPerWeb/data/results/LambdaMart_Re-ranker/ann_LambdaMart_at_100.csv")
report.sort_values(by='mrr@50',  ascending=False).head(5)

In [ ]:
report = create_report(qrels, runs, runs_names, 100,"/Users/user/question-retrieval-KIPerWeb/data/results/LambdaMart_Re-ranker/ann_LambdaMart_at_100.csv")
report.sort_values(by='mrr@100',  ascending=False).head(5)